# Agent LlaMA3-70B Tutorial

Reference
* [Original code](https://github.com/teddylee777/langchain-kr/blob/main/14-Agent/04-Agent-with-various-models.ipynb)
* [LangChain: LLM 서비스를 어떻게 개발할 수 있을까?](https://bigwaveai.tistory.com/76)
* [teddylee777/langchain-kr, github.](https://github.com/teddylee777/langchain-kr/tree/main/14-Agent)
* [llama3로 agent 만드는 방법 + 8B 오픈 모델로 Agent 구성하는 방법, 2024, Youtube](https://youtu.be/04MM0PXv2Fk?si=jFIAZoYCPLRiy1FJ)

In [1]:
!pip install langchain-openai==0.1.7
!pip install langchain==0.2.1
!pip install langchainhub==0.1.16
!pip install langchain-community==0.2.1
!pip install langchain-text-splitters==0.2.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 308.5/308.5 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.6/320.6 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.8/122.8 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.5/142.5 kB 5.0 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 24.0
    Uninstalling packaging-24.0:
      Successfully uninstalled packaging-24.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 973.5/973.5 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
!pip install pypdf==4.2.0
!pip install InstructorEmbedding==1.0.1
!pip install sentence-transformers==2.2.2
!pip install faiss-gpu==1.7.2
!pip install faiss-cpu==1.8.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.4/290.4 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 933.9 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cach

In [3]:
import os

# Tools

```python
# -- Overview
import os
os.environ["TAVILY_API_KEY"] = "<fill the API key>"

from langchain.utilities.tavily_search import TavilySearchAPIWrapper
from langchain_community.tools.tavily_search import TavilySearchResults

from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain.document_loaders import PyPDFLoader
from langchain.tools.retriever import create_retriever_tool

from InstructorEmbedding import INSTRUCTOR
from sentence_transformers import SentenceTransformer
from langchain.embeddings import HuggingFaceInstructEmbeddings



# -- Web search tool
search = TavilySearchAPIWrapper()
tavily_tool = TavilySearchResults(api_wrapper=search)

# -- Embedding model
model_kwargs = {'device': 'cuda'} # Do not include 'token' here.
encode_kwargs = {'normalize_embeddings': True}

embedding_model = HuggingFaceInstructEmbeddings(
                    model_name="hkunlp/instructor-xl",
                    model_kwargs=model_kwargs,
                    encode_kwargs=encode_kwargs,)


# -- PDF 파일 경로 입력 및 로드.
loader = PyPDFLoader("./SPRI_AI_Brief_2024년5월호_F.pdf")   

# -- 텍스트 분할기를 사용하여 문서를 분할.
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)

# -- 문서를 로드하고 분할.
split_docs = loader.load_and_split(text_splitter)

# -- VectorStore를 생성.
vector = FAISS.from_documents(split_docs, embedding_model)


# -- PDf 문서에서 Query 에 대한 관련성 높은 Chunk 를 가져옴.
query = "엔비디아, 차세대 AI 플랫폼 ‘블랙웰(Blackwell)'에 대한 내용을 알려줘"
retriever.get_relevant_documents(query)[0]


# -- retriever tool 생성.
manual = "엔비디아 관련 정보를 PDF 문서에서 검색합니다. '2024년 5월 AI 산업동향' 과 관련된 질문은 이 도구를 사용해야 합니다!"
retriever_tool = create_retriever_tool(
    retriever,
    name="pdf_search",
    description=manual,
)

# tools 리스트 정의
tools = [tavily_tool, retriever_tool]

```

## Search API Tool
https://app.tavily.com/sign-in

* 위 링크에서 API Key 발급
* 현재 [1,000 API calls 에 한해 무료임](https://docs.tavily.com/docs/faq#what-is-tavily-api-pricing).
* https://docs.tavily.com/docs/tavily-api/langchain ;사용 방법

In [43]:
# TAVILY API 세팅
os.environ["TAVILY_API_KEY"] = "<fill the API key>"

from langchain.utilities.tavily_search import TavilySearchAPIWrapper
from langchain_community.tools.tavily_search import TavilySearchResults


search = TavilySearchAPIWrapper()
tavily_tool = TavilySearchResults(api_wrapper=search)

[타빌리(Tavily) Search API](https://tavily.com/) 도구와 관련된 기능을 제공하는 클래스의 인스턴스 초기화.
* 즉, **검색 결과를 처리하는 객체**를 생성함.

## PDF 기반 문서 검색 도구: Retriever


In [5]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain.document_loaders import PyPDFLoader

In [6]:
# from langchain_openai import OpenAIEmbeddings

# -- OpenAIEmbeddings 대체 수단
#    아래 세 패키지는 서로 의존함.
from InstructorEmbedding import INSTRUCTOR
from sentence_transformers import SentenceTransformer
from langchain.embeddings import HuggingFaceInstructEmbeddings

model_kwargs = {'device': 'cuda'} # Do not include 'token' here.
encode_kwargs = {'normalize_embeddings': True}

embedding_model = HuggingFaceInstructEmbeddings(
                    model_name="hkunlp/instructor-xl",
                    model_kwargs=model_kwargs,
                    encode_kwargs=encode_kwargs,)

/usr/local/lib/python3.10/dist-packages/InstructorEmbedding/instructor.py:7: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


.gitattributes:   0%|          | 0.00/1.48k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/270 [00:00<?, ?B/s]

2_Dense/config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/3.15M [00:00<?, ?B/s]

...

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.40k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/461 [00:00<?, ?B/s]

load INSTRUCTOR_Transformer
max_seq_length  512


`OpenAIEmbeddings` 대체 수단:
* [hkunlp/instructor-large, huggingface.](https://huggingface.co/hkunlp/instructor-large/discussions/19)

* [BEST OPEN Alternative to OPENAI's EMBEDDINGs for Retrieval QA: LangChain, 2023](https://youtu.be/ogEalPMUCSY?si=kO0mX4bgjuRWRcYb)

In [7]:
!wget --no-clobber https://raw.githubusercontent.com/DoranLyong/langchain-tutorial-kor/main/Agent/data/SPRI_AI_Brief_2024_May.pdf

# -- PDF 파일 경로 입력 및 로드.
loader = PyPDFLoader("./SPRI_AI_Brief_2024_May.pdf")

--2024-05-29 15:54:21--  https://raw.githubusercontent.com/DoranLyong/langchain-tutorial-kor/main/Agent/data/SPRI_AI_Brief_2024_May.pdf
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5012975 (4.8M) [application/octet-stream]
Saving to: ‘SPRI_AI_Brief_2024_May.pdf’

SPRI_AI_Brief_2024_ 100%[===================>]   4.78M  --.-KB/s    in 0.09s   

2024-05-29 15:54:22 (55.8 MB/s) - ‘SPRI_AI_Brief_2024_May.pdf’ saved [5012975/5012975]



* 예시 pdf: [SPRi 소프트웨어정책연구소](https://spri.kr/posts?code=AI-Brief)

In [8]:
# -- 텍스트 분할기를 사용하여 문서를 분할.
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)

긴 텍스트를 일정한 크기의 청크(chunk)로 분할하기 위한 객체 생성.
* `chunk_size=1000`: 각 청크의 최대 길이를 1000자로 설정. 즉, 텍스트가 1000자 단위로 분할함.
* `chunk_overlap=100`: 각 청크가 겹치는 부분의 길이를 100자로 설정. 이는 청크 간의 일부 중복을 허용하여 **텍스트의 연속성을 유지함**.

In [9]:
# -- PDF 문서를 로드 및 분할.
split_docs = loader.load_and_split(text_splitter)

print(f"Num of chunks: {len(split_docs)}")
print(split_docs[1])

Num of chunks: 43
page_content='2024 년 5월호\nⅠ. 인공지능 산업 동향 브리프\n 1. 정책/법제 \n   ▹ 유엔총회 , AI의 안전한 사용을 위한 결의안을 만장일치로 채택········································· 1\n   ▹ 미국과 영국, AI 모델의 안전 테스트 협력을 위한 MOU 체결·········································· 2\n   ▹ 미국 백악관 예산관리국 , 연방정부의 AI 사용에 관한 정책 발표······································· 3\n   ▹ 캐나다 정부, AI 산업 육성에 2조 4,000 억 원 투자 계획················································· 4\n   ▹ 과기정통부 , AI전략최고위협의회 출범 및 AI 혁신성장과 AI 일상화 추진························· 5\n \n 2. 기업/산업 \n   ▹ 엔비디아 , 차세대 AI 플랫폼 ‘블랙웰 (Blackwell)’ 발표······················································· 6\n   ▹ 일론 머스크의 xAI, ‘그록-1.5’ 및 멀티모달 버전 ‘그록-1.5V’ 공개································ 7\n   ▹ 메타, 오픈소스 LLM ‘라마’의 최신 버전 ‘라마 3’ 공개····················································· 8\n   ▹ 바이두 AI 챗봇 ‘어니봇 ’의 사용자 2억 명 돌파 ································································ 9\n 3. 기술/연구\n   ▹ EU집행위원회 , 연구 활동에서 생성 AI의 책임 있는 사용에 관한 지침 발표················· 10' metadata={'source'

In [10]:
# -- VectorStore를 생성.
#vector = FAISS.from_documents(split_docs, OpenAIEmbeddings())
vector = FAISS.from_documents(split_docs, embedding_model)

`FAISS`는 Facebook AI Research에서 개발한 효율적인 **유사성 검색**과 **클러스터링**을 위한 라이브러리. 위 코드는 문서들을 **벡터화**하고, 이를 FAISS **Vector Store**에 저장하는 과정.
* `split_docs`: 분할된 문서 청크 리스트
* `embedding_model`: 문서를 벡터로 변환하기 위한 임베딩 모델. 문서의 텍스트를 입력받아 고차원 벡터로 변환함.
* 벡터화된 문서를 FAISS Vector Store에 저장.
    * Vector Store: 고차원 벡터들을 효율적으로 저장하고 검색할 수 있도록 설계된 데이터 구조

In [11]:
# -- 벡터스토어를 검색기로 변환.
retriever = vector.as_retriever()

FAISS 벡터스토어 `vector`객체를 **검색기로 변환하는 과정**. 이를 통해 벡터스토어에 저장된 벡터들 중에서 유사한 벡터를 효율적으로 검색할 수 있게 됨.
* `.as_retriever` 메서드는 FAISS 벡터스토어를 검색 기능을 제공하는 객체로 변환함.

In [12]:
# -- PDf 문서에서 Query 에 대한 관련성 높은 Chunk 를 가져옴.
query = "엔비디아, 차세대 AI 플랫폼 ‘블랙웰(Blackwell)'에 대한 내용을 알려줘"
retriever.get_relevant_documents(query)[0]

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


Document(page_content='2024 년 5월호\nⅠ. 인공지능 산업 동향 브리프\n 1. 정책/법제 \n   ▹ 유엔총회 , AI의 안전한 사용을 위한 결의안을 만장일치로 채택········································· 1\n   ▹ 미국과 영국, AI 모델의 안전 테스트 협력을 위한 MOU 체결·········································· 2\n   ▹ 미국 백악관 예산관리국 , 연방정부의 AI 사용에 관한 정책 발표······································· 3\n   ▹ 캐나다 정부, AI 산업 육성에 2조 4,000 억 원 투자 계획················································· 4\n   ▹ 과기정통부 , AI전략최고위협의회 출범 및 AI 혁신성장과 AI 일상화 추진························· 5\n \n 2. 기업/산업 \n   ▹ 엔비디아 , 차세대 AI 플랫폼 ‘블랙웰 (Blackwell)’ 발표······················································· 6\n   ▹ 일론 머스크의 xAI, ‘그록-1.5’ 및 멀티모달 버전 ‘그록-1.5V’ 공개································ 7\n   ▹ 메타, 오픈소스 LLM ‘라마’의 최신 버전 ‘라마 3’ 공개····················································· 8\n   ▹ 바이두 AI 챗봇 ‘어니봇 ’의 사용자 2억 명 돌파 ································································ 9\n 3. 기술/연구\n   ▹ EU집행위원회 , 연구 활동에서 생성 AI의 책임 있는 사용에 관한 지침 발표················· 10', metadata={'source': './SPR

* 사용자가 검색하고자 하는 내용을 포함한 쿼리 문자열을 정의
* `.get_relevant_documents(query)` 메서드는 입력된 쿼리 문자열을 기반으로 관련 문서를 벡터스토어에서 검색함.
* 반환된 결과는 관련 문서들의 리스트
    * `[0]`은 이 리스트에서 첫 번째 문서(=청크)를 선택하는 인덱스 연산자. 즉, 가장 관련성이 높은 첫 번째 문서를 가져옴.

## PDF 도구 생성

In [13]:
# -- langchain 패키지의 tools 모듈에서 retriever 도구를 생성하는 함수를 가져옴.
from langchain.tools.retriever import create_retriever_tool

In [28]:
retriever_tool = create_retriever_tool(
    retriever,
    name="pdf_search",
    description="PDF 문서에서 정보를 검색합니다. '2024년 5월 AI 산업동향' 과 관련된 질문은 이 도구를 사용해야 합니다!",
)

* **검색 도구**를 생성하는 과정. 이 도구는 사용자가 지정한 검색기를 기반으로 특정 작업을 수행하도록 설정됨.
    * `retriever`: 검색기 객체. 유사한 문서를 검색하는 기능을 제공함.
    * `name`: 검색 도구의 이름. 이 이름은 도구를 식별하는 데 사용됨.
    * `description`: 검색 도구에 대한 **사용자 설명**. 이 설명은 도구의 용도와 사용 방법을 설명함. 단, 이 설명으로 도구의 실제 기능이 바뀌지는 않음.

## 도구 목록 정의

In [44]:
# -- tools 리스트 정의.
tools = [tavily_tool, retriever_tool] # [web_search_tool, pdf_search_tool]

# LlaMA3 (JSON Agent)
* 활용 모델: https://github.com/sionic-ai/xionic-ko-llama-3-70b


```python
# == Overview
from langchain_openai import ChatOpenAI
from langchain import hub
from langchain.agents import create_json_chat_agent, AgentExecutor

# -- Model load
llama3 = ChatOpenAI(
    base_url="http://sionic.chat:8001/v1",
    api_key="934c4bbc-c384-4bea-af82-1450d7f8128d",
    model="xionic-ko-llama-3-70b",
    temperature=0.1,
)

# -- Use Prompt for JSON Agent
json_prompt = hub.pull("hwchase17/react-chat-json")

# -- Agent
llama3_agent = create_json_chat_agent(llama3, tools, json_prompt)

llama3_agent_executor = AgentExecutor(
    agent=llama3_agent,
    tools=tools,
    verbose=True,
    handle_parsing_errors=True,
    return_intermediate_steps=True,
)


# -- 검색 결과를 요청 후 질문에 대한 답변 출력.
Question = "2024년 4월 메타에서 공개한 LLM 모델의 버전을 검색해서 결과를 알려주세요."
response = llama3_agent_executor.invoke(
    {"input": Question}
)
print(f'답변: {response["output"]}')
```

In [45]:
from langchain_openai import ChatOpenAI
from langchain import hub
from langchain.agents import create_json_chat_agent, AgentExecutor

## Model load

In [46]:
llama3 = ChatOpenAI(
    base_url="http://sionic.chat:8001/v1",
    api_key="934c4bbc-c384-4bea-af82-1450d7f8128d",
    model="xionic-ko-llama-3-70b",
    temperature=0.1,
)

`ChatOpenAI` 클래스를 사용하여 특정 언어 모델을 설정함.
* `base_url`: 모델이 호스팅되는 서버의 기본 URL
* `api_key`: API 키로, 서버에 접근하기 위한 인증 정보
* `model`: 사용할 모델의 이름
* `temperature`: 생성된 텍스트의 다양성을 조절하는 매개변수

## Prompt template

In [47]:
# Use Prompt Template for JSON Agent
json_prompt = hub.pull("hwchase17/react-chat-json")

# 가져온 JSON 프롬프트를 출력합니다.
print(json_prompt)

input_variables=['agent_scratchpad', 'input', 'tool_names', 'tools'] input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]], 'agent_scratchpad': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]} metadata={'lc_hub_owner': 'hwchase17', 'lc_hub_repo': 'react-chat-json', 'lc_hub_commit_hash': '9c1258e8aa8ce33bebbd62e077c143d0b06c81f3c7de732187ee61c70c1254c7'} messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='Assistant is a large language model trained by Open

https://smith.langchain.com/hub/hwchase17/react-chat-json

`hub.pull` 함수를 사용하여 `"hwchase17/react-chat-json"`이라는 이름의 JSON 프롬프트를 가져오는 과정.


## Build Agent

In [48]:
# Agent
llama3_agent = create_json_chat_agent(llama3, tools, json_prompt)

JSON 형식의 프롬프트를 기반으로 대화형 AI 에이전트를 생성:
* `llama3`: `ChatOpenAI` 객체로, LLaMA 3 모델을 사용
* `tools`: 에이전트가 사용할 수 있는 도구들의 리스트
* `json_prompt`: JSON 형식의 프롬프트로, 에이전트가 대화를 시작하거나 특정 작업을 수행하는 데 필요한 지침을 제공함

In [49]:
llama3_agent_executor = AgentExecutor(
    agent=llama3_agent,
    tools=tools,
    verbose=True,
    handle_parsing_errors=True,
    return_intermediate_steps=True,
)

`AgentExecutor` 클래스를 사용하여 `llama3_agent` 에이전트를 실행할 준비를 함:
* `agent`: 이전에 생성한 에이전트 객체. 대화형 AI 모델로서 주어진 프롬프트와 도구들을 기반으로 작업을 수행함.
* `tools`: 에이전트가 사용할 수 있는 도구들의 리스트. 에이전트가 특정 작업을 수행하거나 정보를 검색하는 데 사용됨.
* `verbose`: `True`이면, 실행 중 자세한 로그를 출력함. 디버깅이나 실행 과정을 상세히 모니터링할 때 유용함.
* `handle_parsing_errors`: `True`이면, **파싱 오류가 발생할 경우 이를 처리**함. 파싱 오류는 일반적으로 입력 데이터가 예상된 형식과 맞지 않을 때 발생할 수 있음. **이 설정은 에이전트가 오류를 무시하고 계속 실행될 수 있도록 함**.
* `return_intermediate_steps`: `True`이면, 에이전트가 작업을 수행하는 동안 **중간 단계를 반환**함. 이를 통해 에이전트의 실행 과정을 더 잘 이해하고 디버깅할 수 있음.

## Invoke

In [50]:
# 검색 결과를 요청 후 질문에 대한 답변 출력.
response = llama3_agent_executor.invoke(
    {"input": "2024년 메타에서 공개한 LLM 모델의 버전을 검색해서 결과를 알려주세요."}
)



> Entering new AgentExecutor chain...
```
{
    "action": "pdf_search",
    "action_input": "2024년 메타 LLM 모델 버전"
}
```SPRi AI Brief |  2024-5 월호
8메타, 오픈소스 LLM ‘라마’의 최신 버전 ‘라마 3’ 공개 
n메타가 매개변수 80억 개와 700억 개의 오픈소스 LLM ‘라마 3’를 공개했으며 , 라마 3는 사전 
훈련과 사후 미세조정 과정을 고도화하여 라마 2보다 성능이 대폭 개선됨
n메타는 페이스북과 인스타그램 등 자사의 서비스 전반에 라마 3 기반의 AI 챗봇 ‘메타 AI’를 
도입하는 한편, 메타 AI를 이용할 수 있는 독립형 웹사이트도 출시 KEY Contents
£차세대 오픈소스 LLM ‘라마 3’, 동급 매개변수의 경쟁 모델보다 우수 
n메타가 2024년 4월 18일 차세대 LLM ‘라마(Llama) 3’의 3개 모델 중 매개변수 80억 개와 700억 
개의 모델을 오픈소스로 공개했으며 , 추후 4,000 억 개 매개변수 모델도 공개 예정 
∙이번에 공개된 모델은 텍스트 기반으로 , 메타는 향후 라마 3에서 다국어와 멀티모달을 지원하고 , 
컨텍스트 창을 확대하며 추론과 코딩 등 전반적인 LLM 기능을 지속적으로 향상할 계획 
n라마 3는 사전 훈련과 사후 미세조정 과정을 고도화하여 라마 2보다 성능이 대폭 개선되었으며 , 
동급 매개변수를 갖는 경쟁 모델의 성능을 능가
∙라마 3는 라마 2보다 7배 더 큰 15조 개 이상의 토큰으로 훈련되었으며 , 일상적 질문부터 
과학·기술·공학·수학(STEM), 코딩, 역사 지식 등 다양한 분야의 데이터셋을 사용해 미세조정을 거침
∙라마 3 70B는 객관식 문제(MMLU) 와 코딩(HumanEval) 성능 벤치마크에서 구글의 ‘제미나이 프로 
1.5’와  앤스로픽의 ‘클로드 3 소네트 (Sonnet)’ 를 능가했으나 , 수학 단어 문제(MATH) 해결이나 
대학원생 수준의 객관식 문제(GPQ

In [51]:
print(response.keys())
print(f'답변: {response["output"]}')

dict_keys(['input', 'output', 'intermediate_steps'])
답변: The latest version of Meta's open-source LLM model is 'Llama 3', which was released in April 2024. It has 80 billion and 700 billion parameters, and is expected to be more powerful than its predecessor, Llama 2.


In [52]:
print(f'답변: {response["intermediate_steps"]}')

답변: [(AgentAction(tool='pdf_search', tool_input='2024년 메타 LLM 모델 버전', log='```\n{\n    "action": "pdf_search",\n    "action_input": "2024년 메타 LLM 모델 버전"\n}\n```'), 'SPRi AI Brief |  2024-5 월호\n8메타, 오픈소스 LLM ‘라마’의 최신 버전 ‘라마 3’ 공개 \nn메타가 매개변수 80억 개와 700억 개의 오픈소스 LLM ‘라마 3’를 공개했으며 , 라마 3는 사전 \n훈련과 사후 미세조정 과정을 고도화하여 라마 2보다 성능이 대폭 개선됨\nn메타는 페이스북과 인스타그램 등 자사의 서비스 전반에 라마 3 기반의 AI 챗봇 ‘메타 AI’를 \n도입하는 한편, 메타 AI를 이용할 수 있는 독립형 웹사이트도 출시 KEY Contents\n£차세대 오픈소스 LLM ‘라마 3’, 동급 매개변수의 경쟁 모델보다 우수 \nn메타가 2024년 4월 18일 차세대 LLM ‘라마(Llama) 3’의 3개 모델 중 매개변수 80억 개와 700억 \n개의 모델을 오픈소스로 공개했으며 , 추후 4,000 억 개 매개변수 모델도 공개 예정 \n∙이번에 공개된 모델은 텍스트 기반으로 , 메타는 향후 라마 3에서 다국어와 멀티모달을 지원하고 , \n컨텍스트 창을 확대하며 추론과 코딩 등 전반적인 LLM 기능을 지속적으로 향상할 계획 \nn라마 3는 사전 훈련과 사후 미세조정 과정을 고도화하여 라마 2보다 성능이 대폭 개선되었으며 , \n동급 매개변수를 갖는 경쟁 모델의 성능을 능가\n∙라마 3는 라마 2보다 7배 더 큰 15조 개 이상의 토큰으로 훈련되었으며 , 일상적 질문부터 \n과학·기술·공학·수학(STEM), 코딩, 역사 지식 등 다양한 분야의 데이터셋을 사용해 미세조정을 거침\n∙라마 3 70B는 객관식 문제(MMLU) 와 코딩(HumanEval) 성능 벤치마크에서 구글의 ‘제미나이 프로 \n1.5’와  앤스로픽의 ‘클로드 3 소네트

` "action": "pdf_search"`
* 앞서 정의한 PDF 검색 도구를 활용했다는 뜻.

In [53]:
# 검색 결과를 요청 후 질문에 대한 답변 출력.
Question = "2024년 메타 LLM 모델의 공개 날짜를 검색해서 결과를 알려주세요."

response = llama3_agent_executor.invoke(
    {"input": Question}
)

print(f'답변: {response["output"]}')



> Entering new AgentExecutor chain...
```
{
    "action": "pdf_search",
    "action_input": "2024년 메타 LLM 모델의 공개 날짜"
}
```SPRi AI Brief |  2024-5 월호
8메타, 오픈소스 LLM ‘라마’의 최신 버전 ‘라마 3’ 공개 
n메타가 매개변수 80억 개와 700억 개의 오픈소스 LLM ‘라마 3’를 공개했으며 , 라마 3는 사전 
훈련과 사후 미세조정 과정을 고도화하여 라마 2보다 성능이 대폭 개선됨
n메타는 페이스북과 인스타그램 등 자사의 서비스 전반에 라마 3 기반의 AI 챗봇 ‘메타 AI’를 
도입하는 한편, 메타 AI를 이용할 수 있는 독립형 웹사이트도 출시 KEY Contents
£차세대 오픈소스 LLM ‘라마 3’, 동급 매개변수의 경쟁 모델보다 우수 
n메타가 2024년 4월 18일 차세대 LLM ‘라마(Llama) 3’의 3개 모델 중 매개변수 80억 개와 700억 
개의 모델을 오픈소스로 공개했으며 , 추후 4,000 억 개 매개변수 모델도 공개 예정 
∙이번에 공개된 모델은 텍스트 기반으로 , 메타는 향후 라마 3에서 다국어와 멀티모달을 지원하고 , 
컨텍스트 창을 확대하며 추론과 코딩 등 전반적인 LLM 기능을 지속적으로 향상할 계획 
n라마 3는 사전 훈련과 사후 미세조정 과정을 고도화하여 라마 2보다 성능이 대폭 개선되었으며 , 
동급 매개변수를 갖는 경쟁 모델의 성능을 능가
∙라마 3는 라마 2보다 7배 더 큰 15조 개 이상의 토큰으로 훈련되었으며 , 일상적 질문부터 
과학·기술·공학·수학(STEM), 코딩, 역사 지식 등 다양한 분야의 데이터셋을 사용해 미세조정을 거침
∙라마 3 70B는 객관식 문제(MMLU) 와 코딩(HumanEval) 성능 벤치마크에서 구글의 ‘제미나이 프로 
1.5’와  앤스로픽의 ‘클로드 3 소네트 (Sonnet)’ 를 능가했으나 , 수학 단어 문제(MATH) 해결이나 
대학원생 수준의 객관식 문제

In [54]:
# 검색 결과를 요청 후 질문에 대한 답변 출력.
Question = "Nvidia가 공개한게 무엇인지 검색해서 결과를 알려주세요."

response = llama3_agent_executor.invoke(
    {"input": Question}
)

print(f'답변: {response["output"]}')



> Entering new AgentExecutor chain...
```
{
    "action": "tavily_search_results_json",
    "action_input": "Nvidia latest announcements"
}
```[{'url': 'https://nvidianews.nvidia.com/news', 'content': 'NVIDIA today reported revenue for the first quarter ended April 28, 2024, of $26.0 billion, up 18% from the previous quarter and up 262% from a year ago. Read Press Release. May 21, 2024.'}, {'url': 'https://nvidianews.nvidia.com/news/latest', 'content': 'What Is a SuperNIC?\nNVIDIA’s New Ethernet Networking Platform for AI Available Soon From Dell Technologies, Hewlett Packard Enterprise, Lenovo\nFrom Guangzhou to Los Angeles, Automakers Dazzle With AI-Powered Vehicles\nAI to See “Major Second Wave,” NVIDIA CEO Says in Fireside Chat With iliad Group Exec\nDropbox and NVIDIA Team to Bring Personalized Generative AI to Millions of Customers\nNVIDIA and Scaleway Speed Development for European Startups and Enterprises\nAI Training AI: GatorTronGPT at the Forefront of University of Florida

In [55]:
# 검색 결과를 요청 후 질문에 대한 답변 출력.
Question = "Nvidia가 무엇에 대해 소개했는지 검색해서 결과를 알려주세요."

response = llama3_agent_executor.invoke(
    {"input": Question}
)

print(f'답변: {response["output"]}')



> Entering new AgentExecutor chain...
```
{
    "action": "tavily_search_results_json",
    "action_input": "Nvidia latest announcements"
}
```[{'url': 'https://www.nvidia.com/en-us/geforce/news/geforce-computex-2022-announcements/', 'content': "At this week's COMPUTEX 2022, NVIDIA made many announcements; everything GeForce is summarized below, and you'll find our full breadth of NVIDIA news in our keynote. So read on to learn about new RTX games, new Reflex games, new GeForce RTX and NVIDIA Studio laptops, and a groundbreaking new 500Hz G-SYNC esports gaming monitor."}, {'url': 'https://nvidianews.nvidia.com/news/latest', 'content': 'What Is a SuperNIC?\nNVIDIA’s New Ethernet Networking Platform for AI Available Soon From Dell Technologies, Hewlett Packard Enterprise, Lenovo\nFrom Guangzhou to Los Angeles, Automakers Dazzle With AI-Powered Vehicles\nAI to See “Major Second Wave,” NVIDIA CEO Says in Fireside Chat With iliad Group Exec\nDropbox and NVIDIA Team to Bring Personalized G

In [57]:
# 검색 결과를 요청 후 질문에 대한 답변 출력.
Question = "엔비디아가 무엇에 대해 소개했는지 PDF에서 검색해서 결과를 알려주세요."

response = llama3_agent_executor.invoke(
    {"input": Question}
)

print(f'답변: {response["output"]}')



> Entering new AgentExecutor chain...
```
{
    "action": "pdf_search",
    "action_input": "엔비디아가 무엇에 대해 소개했는지"
}
```Ⅰ. 인공지능 산업 동향 브리프

업무영역 확대 등으로 신규 일자리가 늘면서 전체 일자리는 큰 변화가 없을 것(75.0%) 으로 예상
∙AI 도입․확대로 인한 일자리 대체 효과가 신규 일자리 창출보다 더 크게 나타나 “전체 일자리가 축소될 
것”이란 응답은 17.9% 였음
<주요 기업의 AI 도입 현황>
☞ 출처 : 한국경영자총협회 , 경총, 「주요 기업 AI 도입 실태 및 인식 조사」 결과 발표, 2024.03.21.

일상에 선도적인 AI 도입을 추진하는 한편, 디지털 심화의 온전한 수용을 위해 지난해 수립한 ‘디지털 
권리장전 ’을 바탕으로 디지털 신질서 정립에 나설 계획
n과기정통부는 AI 일상화를 위한 2024년 국민·산업·공공 프로젝트 추진 계획도 발표했으며 , 일상과 
산업 현장, 정부행정 내 AI 확산에 총 7,102 억원을 투입해 69개의 과제를 추진할 계획
∙국민 일상 속 AI 행복 확산에 755억 원을 투입해 취약계층 ·사회복지 분야의 AI 도입 및 보육·교육· 
문화·주거 등에 AI 확산을 지원
∙공공행정 AI 내재화에 1,157 억 원을 투입해 재난대비와 일상 업무보조 등 공공행정의 AI 활용 촉진으로 
사회현안 해결 지원과 과학적 정부 구현, 대국민 서비스의 질적·양적 제고를 도모
∙AI 시대에 대비해 국민의 AI 활용 역량 제고와 AI 윤리 등 안전한 AI 기반 인프라 확충에 총 
2,309 억원을 투입
☞ 출처:  과학기술정보통신부 , 인공지능 최고위 거버넌스 『AI전략최고위협의회 』 출범, 2024.04.04.

⑦일관성으로 구성됨
∙레드팀 수행 경험 등이 있는 데이터 분야 전문가 , 의료‧금융‧법률 전문가 120여 명으로 구성된 
평가단이 참가자를 평가
n과기정통부는 레드팀 챌린지 결과를 분석하여 생성 AI 분야의 공

## 통찰
* 비슷한 의미를 가진 질문이라도 **들어가는 키워드가 어떤가에 따라** 답변이 다르다
```python
Q1) "Nvidia가 공개한게 무엇인지 검색해서 결과를 알려주세요."
Q2) "Nvidia가 무엇에 대해 소개했는지 검색해서 결과를 알려주세요."
Q3) "엔비디아가 무엇에 대해 소개했는지 PDF에서 검색해서 결과를 알려주세요."
```

* `"action":` 플래그를 보면 "pdf_search"인지 "tavily_search_results_json" 인지 에이전트가 선택해서 동작함을 확인.
    *  메타 관련 질문은 "pdf에서 검색해줘" 라고 말하지 안아도 "pdf_search" 도구를 활용함 (신기하군).

In [62]:
# 검색 결과를 요청 후 질문에 대한 답변 출력.
Question = "Find the latest Microsoft Corporation (MSFT;) stock quote"

response = llama3_agent_executor.invoke(
    {"input": Question}
)

print(f'답변: {response["output"]}')



> Entering new AgentExecutor chain...
```
{
    "action": "tavily_search_results_json",
    "action_input": "Microsoft Corporation (MSFT) stock quote"
}
```[{'url': 'https://www.google.com/finance/quote/MSFT:NASDAQ', 'content': 'Get the latest Microsoft Corp (MSFT) real-time quote, historical performance, charts, and other financial information to help you make more informed trading and investment decisions.'}, {'url': 'https://finance.yahoo.com/quote/MSFT;/', 'content': '174.52. +1.34%. Find the latest Microsoft Corporation (MSFT;) stock quote, history, news and other vital information to help you with your stock trading and investing.'}, {'url': 'https://finance.yahoo.com/quote/MSFT/', 'content': 'Find the latest Microsoft Corporation (MSFT) stock quote, history, news and other vital information to help you with your stock trading and investing.'}, {'url': 'https://www.marketwatch.com/investing/stock/msft', 'content': 'It’s Microsoft vs. Google as the AI Battle Shifts to Your PC De

이건 틀랬네?? 왜지
* 검색 내역에 `content': '174.52. +1.34%. Find the latest Microsoft Corporation (MSFT;) stock quote,` 이 있음
* 그런데 해당 링크의 어떤 부분에서 가져온건지 모르겠네이고 ...


결론: 검색한다고 완전히 정답이 아닐 수 있음

In [63]:
# 검색 결과를 요청 후 질문에 대한 답변 출력.
Question = "마이크로소프트의 현재 주식 가격을 알려줘"

response = llama3_agent_executor.invoke(
    {"input": Question}
)

print(f'답변: {response["output"]}')



> Entering new AgentExecutor chain...
```
{
    "action": "tavily_search_results_json",
    "action_input": "Microsoft current stock price"
}
```[{'url': 'https://www.marketwatch.com/investing/stock/msft', 'content': 'It’s Microsoft vs. Google as the AI Battle Shifts to Your PC Desktop\nNo Headlines Available\nNo Headlines Available\nNo Headlines Available\nWhy Microsoft Stock Is Cheaper Than It Looks\nMicrosoft’s Growth Trajectory and Buy Rating Reinforced by Cloud and AI Advancements\nDollar Decline: Stock Market Rally Explains 70% Of Recent US Dollar Downtrend, Says BofA\n10 Information Technology Stocks Whale Activity Overview\nMSFT Overview\nKey Data\nPerformance\nAnalyst Ratings\nRecent News\nMicrosoft Corp. stock outperforms market on strong trading day\nTechnology\nNvidia Stock Is Cheap on This Metric. The Productivity and Business Processes segment consists of Office Commercial (Office 365 subscriptions, the Office 365 portion of Microsoft 365 Commercial subscriptions, and O

링크 들어가면 가격이 나오는데 ㅎㅎ.
* 정확해지려면 추가적인 무언가가 필요한 듯

In [64]:
# 검색 결과를 요청 후 질문에 대한 답변 출력.
Question = "2024년 마이크로소프트 CEO의 이름을 알려줘"

response = llama3_agent_executor.invoke(
    {"input": Question}
)

print(f'답변: {response["output"]}')



> Entering new AgentExecutor chain...
```
{
    "action": "tavily_search_results_json",
    "action_input": "Who is the CEO of Microsoft in 2024"
}
```[{'url': 'https://en.wikipedia.org/wiki/Satya_Nadella', 'content': 'Later, he received an MBA from the University of Chicago Booth School of Business in 1997.[25][26]\nCareer\nSun Microsystems\nNadella worked at Sun Microsystems as a member of its technology staff before joining Microsoft in 1992.[27]\nMicrosoft\nAt Microsoft, Nadella has led major projects that included the company\'s move to cloud computing and the development of one of the largest cloud infrastructures in the world.[28]\nNadella worked as the senior vice-president of research and development (R&D) for the Online Services Division and vice-president of the Microsoft Business Division. On October 26, 2018, Microsoft acquired GitHub for US$7.5 billion.[53]\nAs of November 2023, Microsoft stock had increased nearly tenfold since Nadella became CEO in 2014, with a 27% an

이건 정답 :)